In [1]:
import json
import requests
import numpy as np
import pandas as pd

A) Use el API de institutions para extraer la lista de instituciones del país, por ejemplo: colombia (mejor si es Europa, por completo de datos )

Para la primera institución con

number_of_papers > 0

Obtenga el valor

legacy_ICN:

In [121]:
Country = "Finland"
URL = "https://inspirehep.net/api/institutions?q={}".format(Country)

In [122]:
institution_link = requests.get(URL)

In [123]:
legacy_ICN = np.array([institution.get('metadata').get('legacy_ICN') for institution in institution_link.json().get('hits').get('hits')])
number_of_papers = np.array([institution.get('metadata').get('number_of_papers') for institution in institution_link.json().get('hits').get('hits')])

legacy_ICN=legacy_ICN[number_of_papers>0][0] #first institution with at least one article
legacy_ICN

'Helsinki, Academy of Finland'

B) Con el API de literatura obtenga el JSON con los artículos de menos de 10 autores usando el "legacy_ICN" de la siguiente manera


https://inspirehep.net/api/literature?sort=mostrecent&page=1&q=aff+Colombia,+U.+Natl.+and+ac+1->+10


aff: usa el valor de legacy_ICN

and: es un operador lógico

ac: establece los autores entre 1 y 10

In [124]:
literature_link ="https://inspirehep.net/api/literature?sort=mostrecent&page=1&q=aff+{}+and+ac+1->+10".format(legacy_ICN.replace(' ','+'))
literature_link = requests.get(literature_link)

C) Para al menos un artículo de esa institución, extraiga el URL del perfil de cada autor de esa institución que se encuentra dentro del campo "authors" en "record" y luego en "$ref". Por ejemplo

In [125]:
articles = [article.get('metadata') for article in literature_link.json().get('hits').get('hits')]
len(articles)

10

In [130]:
#extracting authors for first article
authors0_profile = [author.get('record').get('$ref') for author in articles[3].get('authors')]
authors0_profile

['https://inspirehep.net/api/authors/989957',
 'https://inspirehep.net/api/authors/994387',
 'https://inspirehep.net/api/authors/991666']

D) Con cada uno de los datos del resultado del API para cada perfil construya una tabla con los siguientes columnas (puede que alguno de los datos no esté disponible): 

Nombre Completo

Correo electrónico

posición más reciente (la primera  que aparece en la lista "positions" del JSON) con su correspondiente:
rango 

institución 

fecha de inicio 

fecha de finalización

In [152]:
def get_info(URL):
    info = requests.get(URL).json().get('metadata')

    name = info.get('name').get('value')

    email = info.get('email_addresses')[0].get('value') #first valid email adress
 
    last_position = info.get('positions')[0].get('rank') #last position

    intitution = info.get('positions')[0].get('institution') #institution where they  had their last position

    start = info.get('positions')[0].get('start_date')

    end = info.get('positions')[0].get('end_date')
    
    print('\n Name : {} \n email : {} \n last_position : {} \n institution : {} \n start_date : {} \n end_date : {}\n'.format(name , email, last_position,intitution,start,end))

In [150]:
print("For the first article found: '{}' of the institution: '{}', the authors are".format(articles[0].get('titles')[0].get('title'),legacy_ICN))

for author in authors0_profile:
    get_info(author)

For the first article found: 'Bayesian inference of the fluctuating proton shape in DIS and hadronic collisions' of the institution: 'Helsinki, Academy of Finland', the authors are

 Name : Schiavilla, Rocco 
 email : [{'value': 'schiavil@jlab.org', 'current': True}] 
 last_position : SENIOR 
 institution : Jefferson Lab 
 start_date : None 
 end_date : None


 Name : Pandharipande, Vijay R. 
 email : None 
 last_position : None 
 institution : Illinois U., Urbana 
 start_date : None 
 end_date : None


 Name : Riska, Dan-Olof 
 email : [{'value': 'riska@pcu.helsinki.fi', 'current': False}] 
 last_position : STAFF 
 institution : Helsinki Inst. of Phys. 
 start_date : 2000 
 end_date : None



In [148]:
URL = authors0_profile[1]

info = requests.get(URL).json().get('metadata')

name = info.get('name').get('value')

email = info.get('email_addresses')[0].get('value') #first valid email adress

last_position = info.get('positions')[0].get('rank') #last position

intitution = info.get('positions')[0].get('institution') #institution where they  had their last position

start = info.get('positions')[0].get('start_date')

end = info.get('positions')[0].get('end_date')

TypeError: 'NoneType' object is not subscriptable

In [139]:
info.get('email_addresses')[0]

{'value': 'schiavil@jlab.org', 'current': True}